###Shanksův algoritmus **Baby-Steps Giant-Steps**
Řeší problém diskrétního logaritmu efektivnějším způsobem, než je postupné prohledávání všech kandidátů na exponenty. Představte si, že hledáte jeden konkrétní bod na kruhu, vytvořeném miliony bodů ve stejných vzdálenostech. Místo toho, abyste procházeli každý bod jeden po druhém, použijete strategii.

Základní myšlenka algoritmu spočívá v rozdělení velkého problému na dva menší podproblémy. Máme multiplikativní cyklickou grupu <code>Z<sub>p</sub>*</code>, kde všechny prvky leží na imaginární kružnici. Hledáme exponent `x` v rovnici <code>g<sup>x</sup> ≡ h (mod p)</code>.

Pomocí substituce, kdy hledaný exponent zapíšeme jako `x = im + j` a `m` je přibližně odmocnina z velikosti grupy <code>Z<sub>p</sub>*</code>, převedeme problém s potenciálně miliony kroků na dva seznamy, každý o velikosti zhruba druhé odmocnině velikosti grupy <code>Z<sub>p</sub>\*</code> v každém podproblému.

###Baby-Steps - malé kroky
V první fázi vytváříme seznam základních hodnot. Postupně vypočítáme mocniny generátoru `g` s malými exponenty: <code>g<sup>0</sup>, g<sup>1</sup>, g<sup>2</sup>, ... g<sup>m-1</sup></code>. Tyto hodnoty uložíme do tabulky spolu s odpovídajícími exponenty. Tento krok algoritmu lze přirovnat k vytvoření mapy orientačních bodů na kružnici.

###Giant-Steps - velké kroky
Ve druhé fázi děláme větší skoky po kružnici. Začneme u hledané hodnoty
`h` a postupně se posunujeme velkými kroky zpět pomocí výpočtu <code>h * g<sup>-im</sup> (mod p)</code>. Po každém kroku kontrolujeme, zda jsme nenarazili na některý z orientačních bodů z první fáze. Pokud najdeme shodu, můžeme snadno vypočítat hledaný exponent `x`.

###Pseudokód algoritmu

1. Stanovení velikosti kroku: `m <- |√q|`
2. Inicializuj prázdnou tabulku `T`
3. Baby-Steps: Pro všechna `j`, kde `0 <= j < m`:
- Vypočti <code>g<sup>j</sup> (mod p)</code>,
- Ulož dvojici <code>(j, g<sub>j</sub>)</code> do tabulky `T`,
4. Příprava pro Giant-Steps:
- Vypočti inverzní prvek <code>c <- g<sup>-m</sup> (mod p)</code>,
- Nastav výchozí pozici gamma `γ <- h`,
5. Giant-Steps: Pro všechna `i`, kde `0 <= i < m`:
- Zjisti zda existuje `γ` v tabulce Baby-Steps s odpovídajícím indexem `j`,
- Pokud ano: Vrať hodnotu `x = im + j (mod q)`,
- Pokud ne: Proveď další velký krok,
6. Výsledek: Nalezený exponent `x` splňuje rovnici <code>g<sup>x</sup> ≡ h (mod p)</code>.

###Samostatný úkol: Nyní již znáte všechno potřebné, implementujte algoritmus Baby-Step Giant-Step.

In [ ]:
# IMPLEMENTACE ALGORITMU BABY-STEPS GIANT-STEPS
import math

def baby_step_giant_step(g, h, p):
    n = p - 1
    m = math.isqrt(n) + 1          # Velikost kroku ≈ sqrt(n)

    # ── BABY-STEPS ────────────────────────────────────────────────────────
    # Předpočítáme g^j mod p pro všechna 0 <= j < m a uložíme do slovníku
    baby_steps = {}
    for j in range(m):
        val = pow(g, j, p)
        baby_steps[val] = j         # klíč = hodnota, hodnota = exponent j

    # ── GIANT-STEPS ───────────────────────────────────────────────────────
    # g^(-m) mod p  – Python 3.8+ pow() zvládne záporný exponent přímo
    g_inv_m = pow(g, -m, p)
    gamma = h                       # Začínáme od h, posouváme se velkými kroky

    for i in range(m + 1):
        if gamma in baby_steps:     # Nalezena shoda s Baby-Steps tabulkou
            j = baby_steps[gamma]
            return i * m + j        # x = i*m + j
        gamma = (gamma * g_inv_m) % p

    return None                     # Řešení v dané grupě neexistuje


# ── Ověření na příkladu z přednášky ───────────────────────────────────────
# Z_37* , g=2, h=7  →  očekáváme x=32  (2^32 mod 37 = 7)
g_test, h_test, p_test = 2, 7, 37
x = baby_step_giant_step(g_test, h_test, p_test)
print(f"dlog_{g_test}({h_test}) mod {p_test} = {x}")
print(f"Ověření: {g_test}^{x} mod {p_test} = {pow(g_test, x, p_test)}")

# ── Praktický příklad (stejný jako v učitelově řešení) ────────────────────
g2, h2, p2 = 17, 438, 509
x2 = baby_step_giant_step(g2, h2, p2)
print(f"\ndlog_{g2}({h2}) mod {p2} = {x2}")
print(f"Ověření: {g2}^{x2} mod {p2} = {pow(g2, x2, p2)}")

### Možné řešení algoritmu Baby-Steps Giant-Steps a jeho vizualizace.

In [ ]:
# Algoritmus Sqaure and Multiply (S&M)
def square_and_multiply(base, exp, mod):
    result = 1
    base = base % mod

    while exp > 0:
        if exp % 2 == 1:
            result = (result * base) % mod
        base = (base * base) % mod
        exp //= 2

    return result

# Rozšířený Euklidův algoritmus
def inverse_element(a, m):
    def egcd(a, b):
        if b == 0:
            return a, 1, 0
        g, x1, y1 = egcd(b, a % b)
        x = y1
        y = x1 - (a // b) * y1
        return g, x, y

    g, x, _ = egcd(a, m)
    if g != 1:
        raise Exception("Inverzní prvek neexistuje, čísla jsou nesoudělná.")
    return x % m


def shanks_baby_step_giant_step(g, h, p):
    import math
    n = p - 1
    m = math.isqrt(n) + 1

    # BABY-STEPS
    baby_steps = []
    baby_dict = {}
    for j in range(m):
        val = square_and_multiply(g, j, p)
        baby_steps.append((val, j))
        baby_dict[val] = j

    # GIANT-STEPS
    g_inv = inverse_element(g, p)
    g_inv_m = square_and_multiply(g_inv, m, p)
    gamma = h
    giant_steps = []

    shoda_val = None
    for i in range(m):
        giant_steps.append((gamma, i))
        if gamma in baby_dict:
            j = baby_dict[gamma]
            shoda_val = gamma
            list_table(g, h, p, m, baby_steps, giant_steps, shoda_val)
            return i * m + j, i, j, m
        gamma = (gamma * g_inv_m) % p

In [ ]:
# Metoda zobrazující hodnoty uložené v seznamech kroků
def list_table(g, h, p, m, baby_steps, giant_steps, match_val):
    print("\n" + "=" * 65)
    print(f"{'BABY (g^j mod p)':<30} | {'GIANT (h * g^(-m*i) mod p)':<30} |")
    print("=" * 65)

    max_len = max(len(baby_steps), len(giant_steps))
    for k in range(max_len):
        baby_str = ""
        giant_str = ""

        # BABY STEPS
        if k < len(baby_steps):
            b_val, b_j = baby_steps[k]
            baby_str = f"{b_j, b_val}"
            if b_val == match_val:
                baby_str += " <---"
        else:
            b_val, b_j = (None, None)

        # GIANT STEPS
        if k < len(giant_steps):
            g_val, g_i = giant_steps[k]
            giant_str = f"{g_i, g_val}"
            if g_val == match_val:
                giant_str += " <---"
        else:
            g_val, g_i = (None, None)

        print(f"{baby_str:<30} | {giant_str:<30} |")

    print("=" * 65 + "\n")


# Praktický příklad
g = 17
h = 438
p = 509

x, i, j, m = shanks_baby_step_giant_step(g, h, p)
exponent = i * m + j
print(f"Nalezeno: i = {i}, j = {j} \nx = {i} * {m} + {j} = {exponent} -> {g}^{exponent} ≡ {h} (mod {p})\nx = {x}")

### Měření výkonnosti: závislost doby výpočtu na velikosti grupy

Algoritmus Baby-Step Giant-Step má časovou složitost **O(√n)**, kde `n = p − 1` je řád grupy.
Níže měříme dobu výpočtu pro prvočísla různých velikostí a vykreslíme graf závislosti.

In [ ]:
import time
import random
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# ── Testovací prvočísla různých řádů ──────────────────────────────────────
# Každá dvojice (g, p): g = generátor (základna), p = prvočíslo (modul)
# Pro každé p zvolíme náhodné x a ověříme, že BSGS ho správně najde.
test_primes = [
    (2,    37),
    (2,   509),
    (2,  1009),
    (2,  7919),
    (2, 104_723),
    (2, 1_000_003),
    (2, 9_999_991),
    (2, 99_999_989),
    (2, 999_999_937),
]

bit_lengths  = []   # log2(p)  – "velikost grupy" na ose X
times_ms     = []   # naměřený čas v milisekundách

print(f"{'prvočíslo p':>15} {'bits':>6} {'hledané x':>14} {'nalezené x':>14} {'čas [ms]':>10}")
print("-" * 65)

for g, p in test_primes:
    n = p - 1
    # Zvolíme náhodné x a z něj odvodíme h  →  BSGS musí h ∈ <g> vždy najít
    x_true = random.randint(1, n - 1)
    h = pow(g, x_true, p)

    t0 = time.perf_counter()
    x_found = baby_step_giant_step(g, h, p)
    elapsed_ms = (time.perf_counter() - t0) * 1000

    # Ověření správnosti  (g^x_found ≡ h)
    assert pow(g, x_found, p) == h, f"Chyba pro p={p}!"

    bits = p.bit_length()
    bit_lengths.append(bits)
    times_ms.append(elapsed_ms)

    print(f"{p:>15,} {bits:>6} {x_true:>14,} {x_found:>14,} {elapsed_ms:>10.3f}")

# ── Graf ──────────────────────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(9, 5))

ax.plot(bit_lengths, times_ms, marker="o", linewidth=2,
        markersize=7, color="#2563eb", label="naměřený čas")

# Referenční křivka O(√2^bits) = O(2^(bits/2))
ref_bits = [b for b in range(bit_lengths[0], bit_lengths[-1] + 1)]
scale = times_ms[0] / (2 ** (bit_lengths[0] / 2))
ref_times = [scale * 2 ** (b / 2) for b in ref_bits]
ax.plot(ref_bits, ref_times, linestyle="--", color="#dc2626",
        linewidth=1.5, label=r"referenční $O(\sqrt{p})$")

ax.set_xlabel("Velikost prvočísla p [bity]", fontsize=12)
ax.set_ylabel("Doba výpočtu [ms]", fontsize=12)
ax.set_title("Baby-Step Giant-Step: závislost doby výpočtu na velikosti grupy", fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, linestyle="--", alpha=0.5)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("%.2f"))

plt.tight_layout()
plt.savefig("bsgs_benchmark.png", dpi=150)
plt.show()
print("Graf uložen jako bsgs_benchmark.png")

### Realné dopady prolomení diskrétního logaritmu.

V praxi prolomení DLP vede k okamžité ztrátě dvou pilířů CIA triády a vytváří kritické dopady v oblastech:
- **Webové komunikace** - útoky mužem uprostřed kompromitací protokolů TLS/HTTPS,
- **Digitální identity** - masové falšování digitálních podpisových schémat (software, dokumentů a identit),
- **E-mailové bezpečnosti** - čitelnost obsahu zpráv bez možnosti ověření odesílatele,
- **Infrastruktury** - zneplatnění mechanizmů autentizace (SSH, VPN) a zabezpečení.


Například: **Logjam: TLS vulnerabilities (CVE-2015-4000)**

Útočník, v postavení muže uprostřed (man-in-the-middle) zneužil zranitelnost protokolů TLS 1.2 a starších. Přepisem zpráv pro vyjednávání šifrovacích sad spojení (handshake) zasílaných na server vynutil zranitelnou úroveň zabezpečení protokolu Diffie-Hellman (downgrade) bez odhalení.

Jednorázovým předvýpočtem hodnoty `p` konkrétní 512 bitové multiplikativní grupy <code>Z<sub>p</sub>*</code> oslabených parametrů útočník získal řešení konkrétního problému diskrétního logaritmu. V řádech desítek sekund až minuty tento proces umožňoval získat sdílené tajemství a rekonstruovat klíče relace spojení. Dále dešifrovat komunikaci, nebo v reálném čase manipulovat s přenášenými daty.

Další zdroje ke zranitelnosti Logjam:
- NIST National Vulnerability Database (NVD). _CVE-2015-4000 Detail_. Online. Dostupné z: https://nvd.nist.gov/vuln/detail/cve-2015-4000.
- Red Hat Customer Portal. _Logjam: TLS vulnerabilities (CVE-2015-4000)._ Online. Dostupné z: https://access.redhat.com/articles/1456263.